# Chapter 7 卷积神经网络CNN, Convolutional Neural Network

## 7.1 整体结构
Affine层: Affine->ReLu -> Affine->ReLU -> Affine->ReLU -> Affine->ReLU -> Affine->Softmax->  
CNN: Conv->ReLU->Pooling -> Conv->ReLU->Pooling -> Conv->ReLU -> Affine->ReLU -> Affine->Softmax ->  

## 7.2 卷积层
### 7.2.1 全连接层存在的问题
特征图 feature map  
输入特征图 input feature map  
输出特征图 output feature map  

### 7.2.2 卷积运算
### 7.2.3 填充 padding
### 7.2.4 步幅 stride
输入大小(H,W)  
滤波器大小 (FH, FW)  
输出大小(OH, OW)  
填充P  
步幅S  
OH = (H+2P-FH)/S + 1  
OW = (W+2P-FW)/S + 1
### 7.2.5 3维数据的卷积运算
### 7.2.6 结合方块思考
三维数据(channel, height, width)  
滤波器(output_channel, input_channel, height, width)  
偏置(output_channel, 1,1)   
输出数据(channel, output_channel, output_height, output_weight)  
### 7.2.7 批处理 
(N, C, H, W) * (FN, C, FH, FW) -> (N, FN, OH, OW) + (FN, 1, 1) -> (N, FN, OH, OW)

## 7.3 池化层
Max池化是获取最大值的运算，2 * 2表示目标区域的大小。  
Average池化计算目标区域的平均值。  

## 7.4 卷积层和池化层的实现
### 7.4.1 4维数组

In [3]:
import numpy as np
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [4]:
x[0].shape

(1, 28, 28)

In [5]:
x[1].shape

(1, 28, 28)

### 7.4.2 基于im2col的展开
若按照for循环重复神经网层运算，实现麻烦，处理变慢。  
使用im2col代替for。  

#### im2col函数 image to column
- 将输入的数据展开以适合滤波器(权重)
- 将4维批数据转换为2维
- 因此，要将步幅设置得很大，以使滤波器的应用区域不重叠
- im2col的实现存在比普通的实现消耗更多内存的缺点。

### 7.4.3 卷积层的实现

In [7]:
import sys, os
sys.path.append("./code/common")
from util import im2col

In [9]:
"""
input_data: 由(数据量，通道，高，长)的4维数组构成的输入数据
filter_h: 滤波器的高
filter_w: 滤波器的长
stride: 步幅
pad: 填充
"""
x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1, 5,5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [10]:
class Convolution:
    def __init__(self, w, b, stride=1, pad=0):
        self.w = w
        self.b = b
        self.stride = stride
        self.pad = pad
    def forward(self, x):
        fn, c, fh, fw = self.w.shape
        n, c, h, w = x.shape
        out_h = int(1+(h + 2*self.pad - fh)/self.stride)
        out_w = int(1+(h + 2*self.pad - fw)/self.stride)
        
        col = im2col(x, fh, fw, self.stride, self.pad)
        col_w = self.w.reshape(fn, -1).T
        out = np.dot(col, col_w) + self.b
        
        out = out.reshape(n, out_h, out_w, -1).transpose(0, 3, 1, 2)
        return out

shape: (N, H, W, C) -> Transpose -> (N, C, H, W)  
index: 0, 1, 2, 3   -> Transpose -> 0, 3, 1, 2  

### 7.4.4 池化层的实现

In [11]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    def forward(self, x):
        n, c, h, w = x.shape
        out_h = int(1+(h-self.h)/self.stride)
        out_w = int(1+(w-self.w)/self.stride)
        
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        out = np.max(col, axis=1)
        out = out.reshape(n, out_h, out_w, c).transpose(0, 3, 1, 2)
        return out

## 7.5 CNN的实现

params:  
- input_dim 输入数据的维度: （通道，高，长）
- conv_param 卷积层的超参数(dict)
>* filter_num 滤波器的数量
>* filter_size 滤波器的大小
>* stride 步幅
>* pad 填充
- hidden_size 隐藏层的神经元数量
- output_size 输出层的神经元数量
- weight_int_std 初始化时权重的标准差

In [14]:
class SimpleConvNet:
    def __init__(self, 
                 input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1}, 
                 hidden_size=100, 
                 output_size=10, 
                 weight_init_std=0.01):
        
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad)/filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2)*(conv_output_size/2))
        
        self.params = {}
        self.params['w1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['w2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['w3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.param['w1'],
                                          self.param['b1'],
                                          self.param['stride'],
                                          self.param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['w2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['w3'], self.params['b3'])
        self.last_layer = softmaxwithloss()
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    def gradient(self, x, t):
        self.loss(x, t)
        dout = 1
        dout = self.lastLayer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        grads = {}
        grads['w1'] = self.layers['Conv1'].dw
        grads['b1'] = self.layers['Conv1'].db
        grads['w2'] = self.layers['Affine1'].dw
        grads['b2'] = self.layers['Affine1'].db
        grads['w3'] = self.layers['Affine2'].dw
        grads['b3'] = self.layers['Affine2'].db
        return grads

## 7.6 CNN的可视化
### 7.6.1 第1层权重的可视化
### 7.6.2 基于分层的信息提取

## 7.7 具有代表性的CNN
### 7.7.1 LeNet
激活函数使用sigmoid，CNN使用ReLU；原始的LeNet中使用subsampling缩小中间数据的大小，但现在CNN中使用Max池化。  
### 7.7.2 AlexNet
- 激活函数使用ReLU
- 使用进行局部正规化的LRN层(Local Response Normalization)
- 使用Dropout